In [1]:
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [2]:
df_train = pd.read_csv('database/train.csv',usecols=[1,2,3,4,5], parse_dates=['date'],
                       dtype={'onpromotion':bool},
                      converters = {'unit_sales': lambda u : np.log1p(float(u) if float(u) > 0 else 0)},
                      skiprows=range(1,66458909))

In [3]:
df_train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2016-01-01,25,105574,2.564949,False
1,2016-01-01,25,105575,2.302585,False
2,2016-01-01,25,105857,1.386294,False
3,2016-01-01,25,108634,1.386294,False
4,2016-01-01,25,108701,1.098612,True


In [4]:
df_test = pd.read_csv('database/test.csv', dtype={'onpromotion':bool},
                     parse_dates = ['date'],usecols=[0,1,2,3,4])

In [5]:
df_test.head()

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False


In [6]:
items = pd.read_csv('database/items.csv')

In [7]:
items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [8]:
df_2017 = df_train.loc[df_train.date >= pd.datetime(2017,1,1)]
del df_train
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
35229871,2017-01-01,25,99197,0.693147,False
35229872,2017-01-01,25,103665,2.079442,False
35229873,2017-01-01,25,105574,0.693147,False
35229874,2017-01-01,25,105857,1.609438,False
35229875,2017-01-01,25,106716,1.098612,False


In [14]:
df_store = pd.read_csv('database/stores.csv')
df_store['big_city'] = False
df_store['middle_city'] = False
df_store['little_city'] = False

citys = pd.read_csv('database/city_population.csv')
big_citys = citys[citys['2017 Population'] >= 1000000]['Name'].values
middle_citys = citys.loc[(citys['2017 Population'] >= 100000) & (citys['2017 Population'] < 1000000)]['Name'].values

df_store['big_city'] = df_store['city'].map(lambda u :True if u in big_citys else False).astype(bool)
df_store['middle_city'] = df_store['city'].map(lambda u: True if u in middle_citys else False).astype(bool)
df_store['little_city'] = df_store['city'].map(lambda u: True if u not in big_citys and u not in middle_citys else False).astype(bool)

In [15]:
df_store = df_store[['store_nbr', 'big_city','middle_city','little_city']]

In [16]:
df_store.head()

,store_nbr,big_city,middle_city,little_city
0,1,True,False,False
1,2,True,False,False
2,3,True,False,False
3,4,True,False,False
4,5,False,True,False


In [17]:
big_city_dict = {}
mid_city_dict = {}
lit_city_dict = {}
for i,j in zip(df_store['store_nbr'], df_store['big_city']):
    big_city_dict[i] = j
for i,j in zip(df_store['store_nbr'], df_store['middle_city']):
    mid_city_dict[i] = j
for i,j in zip(df_store['store_nbr'], df_store['little_city']):
    lit_city_dict[i] = j

In [18]:
city_2017_train = pd.merge(df_2017, df_store, how='left', on=['store_nbr'])
city_2017_test = pd.merge(df_test, df_store, how='left', on=['store_nbr'])
#city_2017_train

In [19]:
bc_2017_train = city_2017_train.set_index(['store_nbr', 'item_nbr', 'date'])[['big_city']].unstack(level=-1)
mc_2017_train = city_2017_train.set_index(['store_nbr', 'item_nbr', 'date'])[['middle_city']].unstack(level=-1)
lc_2017_train = city_2017_train.set_index(['store_nbr', 'item_nbr', 'date'])[['little_city']].unstack(level=-1)
bc_2017_train.columns = bc_2017_train.columns.get_level_values(1)
mc_2017_train.columns = mc_2017_train.columns.get_level_values(1)
lc_2017_train.columns = lc_2017_train.columns.get_level_values(1)

In [20]:
ind = list(set(bc_2017_train.index.get_level_values(0)))
for i in ind:
    bc_2017_train.loc[i] = big_city_dict[i]
    
ind = list(set(mc_2017_train.index.get_level_values(0)))
for i in ind:
    mc_2017_train.loc[i] = mid_city_dict[i]
    
ind = list(set(lc_2017_train.index.get_level_values(0)))
for i in ind:
    lc_2017_train.loc[i] = lit_city_dict[i]

In [21]:
bc_2017_test = city_2017_test.set_index(['store_nbr', 'item_nbr', 'date'])[['big_city']].unstack(level=-1)
mc_2017_test = city_2017_test.set_index(['store_nbr', 'item_nbr', 'date'])[['middle_city']].unstack(level=-1)
lc_2017_test = city_2017_test.set_index(['store_nbr', 'item_nbr', 'date'])[['little_city']].unstack(level=-1)

bc_2017_test.columns = bc_2017_test.columns.get_level_values(1)
mc_2017_test.columns = mc_2017_test.columns.get_level_values(1)
lc_2017_test.columns = lc_2017_test.columns.get_level_values(1)

In [22]:
ind = list(set(bc_2017_test.index.get_level_values(0)))
for i in ind:
    bc_2017_test.loc[i] = big_city_dict[i]
    
ind = list(set(mc_2017_test.index.get_level_values(0)))
for i in ind:
    mc_2017_test.loc[i] = mid_city_dict[i]
    
ind = list(set(lc_2017_test.index.get_level_values(0)))
for i in ind:
    lc_2017_test.loc[i] = lit_city_dict[i]

In [23]:
bc_2017_test = bc_2017_test.reindex(bc_2017_train.index).fillna(False) # 肯定会丢掉了很多train里没有的商品
bc_2017 = pd.concat([bc_2017_train, bc_2017_test], axis=1)

mc_2017_test = mc_2017_test.reindex(mc_2017_train.index).fillna(False)
mc_2017 = pd.concat([mc_2017_train, mc_2017_test], axis=1)

lc_2017_test = lc_2017_test.reindex(lc_2017_train.index).fillna(False)
lc_2017 = pd.concat([lc_2017_train, lc_2017_test], axis=1)

In [24]:
bc_2017

date               2017-01-01 2017-01-02 2017-01-03 2017-01-04 2017-01-05  \
store_nbr item_nbr                                                          
1         96995          True       True       True       True       True   
          99197          True       True       True       True       True   
          103520         True       True       True       True       True   
          103665         True       True       True       True       True   
          105574         True       True       True       True       True   
          105575         True       True       True       True       True   
          105577         True       True       True       True       True   
          105693         True       True       True       True       True   
          105737         True       True       True       True       True   
          105857         True       True       True       True       True   
          106716         True       True       True       True       True   
          108079         True       True       True       True       True   
          108634         True       True       True       True       True   
          108696         True       True       True       True       True   
          108698         True       True       True       True       True   
          108701         True       True       True       True       True   
          108786         True       True       True       True       True   
          108797         True       True       True       True       True   
          108831         True       True       True       True       True   
          108862         True       True       True       True       True   
          108952         True       True       True       True       True   
          111223         True       True       True       True       True   
          111397         True       True       True       True       True   
          112830         True       True       True       True       True   
          114778         True       True       True       True       True   
          114790         True       True       True       True       True   
          114799         True       True       True       True       True   
          114800         True       True       True       True       True   
          115267         True       True       True       True       True   
          115611         True       True       True       True       True   
...                       ...        ...        ...        ...        ...   
54        2081056       False      False      False      False      False   
          2081064       False      False      False      False      False   
          2081095       False      False      False      False      False   
          2081142       False      False      False      False      False   
          2081161       False      False      False      False      False   
          2083837       False      False      False      False      False   
          2083859       False      False      False      False      False   
          2084278       False      False      False      False      False   
          2084557       False      False      False      False      False   
          2086882       False      False      False      False      False   
          2087374       False      False      False      False      False   
          2087409       False      False      False      False      False   
          2087543       False      False      False      False      False   
          2087567       False      False      False      False      False   
          2087933       False      False      False      False      False   
          2087978       False      False      False      False      False   
          2088922       False      False      False      False      False   
          2089036       False      False      False      False      False   
          2089339       False      False      False      False      False

In [25]:
del bc_2017_train, bc_2017_test
del mc_2017_train, mc_2017_test
del lc_2017_train, lc_2017_test

## 处理节假日信息

In [26]:
df_holiday = pd.read_csv('database/holidays_events.csv', 
                        parse_dates = ['date'],
                        dtype={'transferred':bool})

In [27]:
holiday_2017 = df_holiday.loc[df_holiday.date >= pd.datetime(2017,1,1)]
holiday_2017 = holiday_2017.loc[holiday_2017['transferred'] == False]

firstday = date(2017,1,1)
endday = date(2017,9,1)
periods = endday - firstday
all_days = pd.date_range(firstday, periods=periods.days, freq='D')

In [28]:
weekend_tmp = map(lambda day: True if(day.dayofweek >=5) else False, all_days)
weekend = []
for i in weekend_tmp:
    weekend.append(i)   
df_weekend = pd.DataFrame({'date':all_days, 'weekend_or_holiday': weekend })

In [29]:
tmp = holiday_2017['date'].values
for i in range(len(tmp)):
    df_weekend.loc[df_weekend['date'] == tmp[i], 'weekend_or_holiday'] = True
df_weekend_and_holiday = df_weekend

In [30]:
hw_2017_train = pd.merge(df_2017, df_weekend_and_holiday, how='left', on=['date'])
hw_2017_train = hw_2017_train.set_index(['store_nbr', 'item_nbr', 'date'])[['weekend_or_holiday']].unstack(level=-1)
hw_2017_train.columns = hw_2017_train.columns.get_level_values(1)

In [31]:
holiday_zip = zip(df_weekend['date'],df_weekend['weekend_or_holiday'])
tmp_dict = {}
for d,h in holiday_zip:
    tmp_dict[d] = h

In [32]:
columns = hw_2017_train.columns
for i in columns:
    hw_2017_train[i] = tmp_dict[i]

In [33]:
hw_2017_train

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995           True        True       False       False   
          99197           True        True       False       False   
          103520          True        True       False       False   
          103665          True        True       False       False   
          105574          True        True       False       False   
          105575          True        True       False       False   
          105577          True        True       False       False   
          105693          True        True       False       False   
          105737          True        True       False       False   
          105857          True        True       False       False   
          106716          True        True       False       False   
          108079          True        True       False       False   
          108634          True        True       False       False   
          108696          True        True       False       False   
          108698          True        True       False       False   
          108701          True        True       False       False   
          108786          True        True       False       False   
          108797          True        True       False       False   
          108831          True        True       False       False   
          108862          True        True       False       False   
          108952          True        True       False       False   
          111223          True        True       False       False   
          111397          True        True       False       False   
          112830          True        True       False       False   
          114778          True        True       False       False   
          114790          True        True       False       False   
          114799          True        True       False       False   
          114800          True        True       False       False   
          115267          True        True       False       False   
          115611          True        True       False       False   
...                        ...         ...         ...         ...   
54        2081056         True        True       False       False   
          2081064         True        True       False       False   
          2081095         True        True       False       False   
          2081142         True        True       False       False   
          2081161         True        True       False       False   
          2083837         True        True       False       False   
          2083859         True        True       False       False   
          2084278         True        True       False       False   
          2084557         True        True       False       False   
          2086882         True        True       False       False   
          2087374         True        True       False       False   
          2087409         True        True       False       False   
          2087543         True        True       False       False   
          2087567         True        True       False       False   
          2087933         True        True       False       False   
          2087978         True        True       False       False   
          2088922         True        True       False       False   
          2089036         True        True       False       False   
          2089339         True        True       False       False   
          2099230         True        True       False       False   
          2101795         True        True       False       False   
          2103250         True        True       False       False   
          2105347         True        True       False       False   
          2106464         True        True       False       False   
          

In [34]:
hw_2017_train.columns

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',
               '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
               '2017-08-14', '2017-08-15'],
              dtype='datetime64[ns]', name='date', length=227, freq=None)

In [35]:
hw_2017_test = pd.merge(df_test, df_weekend_and_holiday, how='left', on=['date'])
hw_2017_test = hw_2017_test.set_index(['store_nbr', 'item_nbr', 'date'])[['weekend_or_holiday']].unstack(level=-1)
hw_2017_test.columns = hw_2017_test.columns.get_level_values(1)

In [36]:
columns = hw_2017_test.columns
for i in columns:
    hw_2017_test[i] = tmp_dict[i]
hw_2017_test = hw_2017_test.reindex(hw_2017_train.index)

In [37]:
hw_2017 = pd.concat([hw_2017_train, hw_2017_test], axis=1)
del hw_2017_train, hw_2017_test
hw_2017

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995           True        True       False       False   
          99197           True        True       False       False   
          103520          True        True       False       False   
          103665          True        True       False       False   
          105574          True        True       False       False   
          105575          True        True       False       False   
          105577          True        True       False       False   
          105693          True        True       False       False   
          105737          True        True       False       False   
          105857          True        True       False       False   
          106716          True        True       False       False   
          108079          True        True       False       False   
          108634          True        True       False       False   
          108696          True        True       False       False   
          108698          True        True       False       False   
          108701          True        True       False       False   
          108786          True        True       False       False   
          108797          True        True       False       False   
          108831          True        True       False       False   
          108862          True        True       False       False   
          108952          True        True       False       False   
          111223          True        True       False       False   
          111397          True        True       False       False   
          112830          True        True       False       False   
          114778          True        True       False       False   
          114790          True        True       False       False   
          114799          True        True       False       False   
          114800          True        True       False       False   
          115267          True        True       False       False   
          115611          True        True       False       False   
...                        ...         ...         ...         ...   
54        2081056         True        True       False       False   
          2081064         True        True       False       False   
          2081095         True        True       False       False   
          2081142         True        True       False       False   
          2081161         True        True       False       False   
          2083837         True        True       False       False   
          2083859         True        True       False       False   
          2084278         True        True       False       False   
          2084557         True        True       False       False   
          2086882         True        True       False       False   
          2087374         True        True       False       False   
          2087409         True        True       False       False   
          2087543         True        True       False       False   
          2087567         True        True       False       False   
          2087933         True        True       False       False   
          2087978         True        True       False       False   
          2088922         True        True       False       False   
          2089036         True        True       False       False   
          2089339         True        True       False       False   
          2099230         True        True       False       False   
          2101795         True        True       False       False   
          2103250         True        True       False       False   
          2105347         True        True       False       False   
          2106464         True        True       False       False   
          

## 处理促销信息

In [38]:
df_test = df_test.set_index(['store_nbr', 'item_nbr', 'date'])

In [39]:
promo_2017_train = df_2017.set_index(['store_nbr','item_nbr','date'])[['onpromotion']].unstack(level=-1).fillna(False)
promo_2017_train

onpromotion                                              \
date                2017-01-01 2017-01-02 2017-01-03 2017-01-04 2017-01-05   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   
          105574         False      False       True      False      False   
          105575         False      False      False      False      False   
          105577         False      False      False      False      False   
          105693         False      False      False      False      False   
          105737         False      False      False      False      False   
          105857         False      False      False      False      False   
          106716         False      False      False      False      False   
          108079         False      False      False      False      False   
          108634         False      False      False      False      False   
          108696         False      False      False      False      False   
          108698         False      False      False      False      False   
          108701         False      False      False      False      False   
          108786         False      False      False      False      False   
          108797         False      False      False      False      False   
          108831         False      False      False      False      False   
          108862         False      False      False      False      False   
          108952         False      False      False      False      False   
          111223         False      False      False      False      False   
          111397         False      False      False      False      False   
          112830         False      False      False      False      False   
          114778         False      False      False      False      False   
          114790         False      False      False      False      False   
          114799         False      False      False      False      False   
          114800         False      False      False      False      False   
          115267         False      False      False      False      False   
          115611         False      False      False      False      False   
...                        ...        ...        ...        ...        ...   
54        2081056        False      False      False      False      False   
          2081064        False      False      False      False      False   
          2081095        False      False      False      False      False   
          2081142        False      False      False      False      False   
          2081161        False      False      False      False      False   
          2083837        False      False      False      False      False   
          2083859        False      False      False      False      False   
          2084278        False      False      False      False      False   
          2084557        False      False      False      False      False   
          2086882        False      False      False      False      False   
          2087374        False      False      False      False      False   
          2087409        False      False      False      False      False   
          2087543        False      False      False      False      False   
          2087567        False      False      False      False      False   
          2087933        False      False      False      False      False   
          2087978        False      False      False      False      False   
          2088922        False      False      False      False      False   
          2089036        False      False

In [40]:
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_train.columns

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',
               '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
               '2017-08-14', '2017-08-15'],
              dtype='datetime64[ns]', name='date', length=227, freq=None)

In [41]:
promo_2017_test = df_test[['onpromotion']].unstack(level=-1).fillna(False)
promo_2017_test.head()

onpromotion                                              \
date                2017-08-16 2017-08-17 2017-08-18 2017-08-19 2017-08-20   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103501         False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   

                                                                           \
date               2017-08-21 2017-08-22 2017-08-23 2017-08-24 2017-08-25   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

                                                                           \
date               2017-08-26 2017-08-27 2017-08-28 2017-08-29 2017-08-30   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

                               
date               2017-08-31  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103501        False  
          103520        False  
          103665        False

In [42]:
promo_2017_test.columns

MultiIndex(levels=[['onpromotion'], [2017-08-16 00:00:00, 2017-08-17 00:00:00, 2017-08-18 00:00:00, 2017-08-19 00:00:00, 2017-08-20 00:00:00, 2017-08-21 00:00:00, 2017-08-22 00:00:00, 2017-08-23 00:00:00, 2017-08-24 00:00:00, 2017-08-25 00:00:00, 2017-08-26 00:00:00, 2017-08-27 00:00:00, 2017-08-28 00:00:00, 2017-08-29 00:00:00, 2017-08-30 00:00:00, 2017-08-31 00:00:00]],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]],
           names=[None, 'date'])

In [43]:
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test.columns

DatetimeIndex(['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
               '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
               '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
               '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31'],
              dtype='datetime64[ns]', name='date', freq=None)

In [44]:
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

In [45]:
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

In [46]:
del promo_2017_test, promo_2017_train

In [47]:
promo_2017.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False        True       False   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False        True       False       False   

date                2017-01-09  2017-01-10     ...      2017-08-22  \
store_nbr item_nbr                             ...                   
1         96995          False       False     ...           False   
          99197          False       False     ...           False   
          103520         False       False     ...           False   
          103665         False       False     ...           False   
          105574         False       False     ...           False   

date                2017-08-23  2017-08-24  2017-08-25  2017-08-26  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-27  2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                              
1         96995          False       False       False       False       False  
          99197          False       False       False       False       False  
          103520         False       False       False       False       False  
          103665         False       False       False       False       False  
          105574         False       False       False       False       False  

[5 rows x 243 columns]

In [48]:
promo_2017.columns

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2017-08-22', '2017-08-23', '2017-08-24', '2017-08-25',
               '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29',
               '2017-08-30', '2017-08-31'],
              dtype='datetime64[ns]', name='date', length=243, freq=None)

In [49]:
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
35229871,2017-01-01,25,99197,0.693147,False
35229872,2017-01-01,25,103665,2.079442,False
35229873,2017-01-01,25,105574,0.693147,False
35229874,2017-01-01,25,105857,1.609438,False
35229875,2017-01-01,25,106716,1.098612,False


In [50]:
df_2017 = df_2017.set_index(['store_nbr', 'item_nbr', 'date'])[['unit_sales']].unstack(level=-1).fillna(0)
df_2017.head()

unit_sales                                              \
date               2017-01-01 2017-01-02 2017-01-03 2017-01-04 2017-01-05   
store_nbr item_nbr                                                          
1         96995           0.0   0.000000   0.000000   0.000000   0.000000   
          99197           0.0   0.000000   1.386294   0.693147   0.693147   
          103520          0.0   0.693147   1.098612   0.000000   1.098612   
          103665          0.0   0.000000   0.000000   1.386294   1.098612   
          105574          0.0   0.000000   1.791759   2.564949   2.302585   

                                                                           \
date               2017-01-06 2017-01-07 2017-01-08 2017-01-09 2017-01-10   
store_nbr item_nbr                                                          
1         96995      0.000000   0.000000   0.000000   0.000000   0.000000   
          99197      0.693147   1.098612   0.000000   0.000000   0.693147   
          103520     1.386294   0.693147   0.000000   0.693147   0.693147   
          103665     1.098612   0.693147   1.098612   0.000000   2.079442   
          105574     1.945910   1.609438   1.098612   1.386294   2.302585   

                      ...                                                  \
date                  ...     2017-08-06 2017-08-07 2017-08-08 2017-08-09   
store_nbr item_nbr    ...                                                   
1         96995       ...       1.098612   1.098612   0.000000   0.000000   
          99197       ...       0.000000   1.098612   0.000000   1.098612   
          103520      ...       0.000000   0.000000   1.386294   0.000000   
          103665      ...       0.693147   1.098612   0.000000   2.079442   
          105574      ...       0.000000   1.791759   2.079442   1.945910   

                                                                           \
date               2017-08-10 2017-08-11 2017-08-12 2017-08-13 2017-08-14   
store_nbr item_nbr                                                          
1         96995      0.693147   0.000000   0.000000   0.000000   0.000000   
          99197      0.000000   0.000000   0.000000   0.000000   0.000000   
          103520     1.386294   0.693147   0.693147   0.693147   0.000000   
          103665     2.302585   1.098612   0.000000   0.000000   0.693147   
          105574     2.397895   1.791759   1.791759   0.000000   1.386294   

                               
date               2017-08-15  
store_nbr item_nbr             
1         96995      0.000000  
          99197      0.000000  
          103520     0.000000  
          103665     0.693147  
          105574     1.609438  

[5 rows x 227 columns]

In [51]:
df_2017.columns = df_2017.columns.get_level_values(1)

## 处理商品信息

In [52]:
items['family_nbr'] = items['family'].astype('category').cat.codes
items['class_nbr'] = items['class'].astype('category').cat.codes

In [55]:
items.head()

,family,class,perishable,family_nbr,class_nbr
item_nbr,,,,,
96995,GROCERY I,1093,0,12,64
99197,GROCERY I,1067,0,12,44
103501,CLEANING,3008,0,7,217
103520,GROCERY I,1028,0,12,17
103665,BREAD/BAKERY,2712,1,5,187


In [56]:
items = items.set_index('item_nbr')

In [58]:
len(items)

4100

In [59]:
items = items.reindex(df_2017.index.get_level_values(1))

In [60]:
len(items)

167515

## 处理商店信息

In [70]:
df_store = pd.read_csv('database/stores.csv')
df_store['city_nbr'] = df_store['city'].astype('category').cat.codes
df_store['state_nbr'] = df_store['state'].astype('category').cat.codes
df_store['type_nbr'] = df_store['type'].astype('category').cat.codes
df_store = df_store.reindex(df_2017.index.get_level_values(0))
df_store.head()

,store_nbr,city,state,type,cluster,city_nbr,state_nbr,type_nbr
store_nbr,,,,,,,,
1,2.0,Quito,Pichincha,D,13.0,18.0,12.0,3.0
1,2.0,Quito,Pichincha,D,13.0,18.0,12.0,3.0
1,2.0,Quito,Pichincha,D,13.0,18.0,12.0,3.0
1,2.0,Quito,Pichincha,D,13.0,18.0,12.0,3.0
1,2.0,Quito,Pichincha,D,13.0,18.0,12.0,3.0


## 提取数据

In [71]:
from datetime import timedelta, date
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [75]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        
        "city_nbr":df_store['city_nbr'].values,
        "state_nbr":df_store['state_nbr'].values,
        "type_nbr":df_store['type_nbr'].values,
        "cluster":df_store['cluster'].values,
        
        "perishable":items['perishable'].values,
        "item_family_nbr":items['family_nbr'].values,
        "item_class_nbr": items['class_nbr'].values,    
        
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        #"weekend_and_holiday":get_timespan(hw_2017, t2017, 0, 1).fillna(0).values.astype(np.uint8).ravel(),
        "restday_2_2017": get_timespan(hw_2017, t2017, 2, 2).sum(axis=1).values,
        "restday_7_2017": get_timespan(hw_2017, t2017, 7, 7).sum(axis=1).values,            
        "big_city":get_timespan(bc_2017, t2017, 0, 1).sum(axis=1).values.ravel(),
        "mid_city": get_timespan(mc_2017, t2017, 0, 1).sum(axis=1).values.ravel(),
        "lit_city": get_timespan(lc_2017, t2017, 0, 1).sum(axis=1).values.ravel(),
         
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_5_2017": get_timespan(df_2017, t2017, 5, 5).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "mean_90_2017": get_timespan(df_2017, t2017, 90, 90).mean(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        
        "promo_7_2017": get_timespan(promo_2017, t2017, 7, 7).sum(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_30_2017": get_timespan(promo_2017, t2017, 30, 30).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_90_2017": get_timespan(promo_2017, t2017, 90, 90).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
        X['mean_12_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values

    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [76]:
print("Preparing dataset...")
t2017 = date(2017, 5, 31)
X_l, y_l = [], []
for i in range(6): # 6
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

Preparing dataset...


In [77]:
X_train

,big_city,city_nbr,cluster,day_1_2017,item_class_nbr,item_family_nbr,lit_city,mean_140_2017,mean_14_2017,mean_30_2017,...,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
store_nbr,,,,,,,,,,,,,,,,,,,,,
1,1.0,18.0,13.0,0.000000,64,12,0.0,0.070156,0.148532,0.138629,...,0,0,0,0,0,0,0,0,0,0
1,1.0,18.0,13.0,0.000000,44,12,0.0,0.134989,0.511931,0.381457,...,0,0,0,0,0,0,0,0,0,0
1,1.0,18.0,13.0,0.000000,17,12,0.0,0.712362,0.667989,0.868856,...,0,0,0,0,0,0,0,0,0,0
1,1.0,18.0,13.0,0.000000,187,5,0.0,1.015355,0.866918,0.956552,...,0,0,0,0,0,0,0,0,0,0
1,1.0,18.0,13.0,1.098612,31,12,0.0,1.805308,1.556041,1.774012,...,0,0,0,0,0,0,0,0,0,0
1,1.0,18.0,13.0,2.302585,31,12,0.0,2.272804,2.066668,2.151082,...,0,0,0,0,0,0,0,0,0,0
1,1.0,18.0,13.0,0.000000,31,12,0.0,0.547424,0.375535,0.608580,...,0,0,0,0,0,0,0,0,0,0
1,1.0,18.0,13.0,0.000000,22,12,0.0,0.117239,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,1.0,18.0,13.0,0.693147,30,12,0.0,0.835684,1.034304,0.905477,...,0,0,0,0,0,0,0,0,0,0


In [78]:
print("Training and predicting models...")
params = {
    'num_leaves': 63, # 31
    'objective': 'regression',
    'min_data_in_leaf': 300,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2, # 2
    'metric': 'l2',
    'num_threads': 5
}

MAX_ROUNDS = 500
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=100
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

Training and predicting models...
Step 1


/home/kylin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.331522	valid_1's l2: 0.318252
[200]	training's l2: 0.306346	valid_1's l2: 0.29738
[300]	training's l2: 0.301711	valid_1's l2: 0.294366
[400]	training's l2: 0.299374	valid_1's l2: 0.292878
[500]	training's l2: 0.29783	valid_1's l2: 0.292107
mean_7_2017: 8591623.21
mean_14_2017: 5309992.30
mean_5_2017: 1016948.17
promo_0: 493585.45
mean_30_2017: 492324.43
day_1_2017: 419292.94
mean_20_dow0_2017: 266102.56
mean_4_dow0_2017: 255786.92
mean_3_2017: 193560.79
mean_12_dow0_2017: 170523.06
promo_7_2017: 130377.30
mean_60_2017: 85036.17
item_family_nbr: 35652.55
promo_14_2017: 29669.50
promo_7: 27950.12
mean_90_2017: 27375.37
mean_4_dow5_2017: 27026.03
item_class_nbr: 26439.51
promo_30_2017: 17368.41
mean_140_2017: 17318.41
promo_140_2017: 14449.92
promo_90_2017: 10453.92
restday_7_2017: 10024.48
type_nbr: 8911.92
promo_14: 8781.86
promo_60_2017: 8780.22
mean_4_dow6_2017: 8557.81
mean_12_dow2_2017: 8555.67
mean

Step 6
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.38391	valid_1's l2: 0.393777
[200]	training's l2: 0.359824	valid_1's l2: 0.369643
[300]	training's l2: 0.35381	valid_1's l2: 0.36627
[400]	training's l2: 0.350861	valid_1's l2: 0.364976
[500]	training's l2: 0.349022	valid_1's l2: 0.364586
mean_14_2017: 5656773.68
mean_30_2017: 4726233.33
mean_7_2017: 2339429.00
mean_3_2017: 944097.43
mean_60_2017: 831830.13
mean_5_2017: 435374.42
mean_12_dow5_2017: 382424.56
promo_5: 378674.20
mean_4_dow5_2017: 260108.73
mean_20_dow5_2017: 253311.01
promo_3: 49404.44
promo_7_2017: 47938.63
restday_7_2017: 43013.67
promo_14_2017: 41939.54
promo_7: 33637.30
promo_30_2017: 33538.71
item_class_nbr: 33201.36
promo_6: 32702.18
mean_4_dow6_2017: 31069.91
restday_2_2017: 30368.19
mean_90_2017: 28678.72
mean_12_dow6_2017: 28570.77
item_family_nbr: 27895.62
day_1_2017: 23051.62
mean_20_dow6_2017: 15917.34
mean_140_2017: 13585.59
promo_60_2017: 11883.15
promo_140_2017: 107

Step 11
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.407643	valid_1's l2: 0.402343
[200]	training's l2: 0.380626	valid_1's l2: 0.380881
[300]	training's l2: 0.373977	valid_1's l2: 0.378078
[400]	training's l2: 0.369993	valid_1's l2: 0.377096
[500]	training's l2: 0.367277	valid_1's l2: 0.376401
mean_30_2017: 6134682.03
mean_14_2017: 3530569.72
mean_60_2017: 2415881.32
mean_7_2017: 2055695.26
mean_5_2017: 1368184.93
mean_12_dow3_2017: 1170879.33
mean_4_dow3_2017: 855742.17
promo_10: 498741.56
mean_20_dow3_2017: 412684.96
mean_4_dow4_2017: 51835.39
item_class_nbr: 42661.48
promo_30_2017: 41008.88
promo_7_2017: 40622.96
mean_4_dow2_2017: 39827.20
item_family_nbr: 39126.55
promo_14: 31367.85
promo_14_2017: 31340.33
restday_7_2017: 30436.99
promo_7: 29048.45
mean_3_2017: 25682.71
mean_12_dow2_2017: 25393.93
promo_12: 24645.11
promo_11: 21103.26
promo_9: 19556.78
mean_90_2017: 19478.51
promo_13: 18491.67
promo_90_2017: 17461.10
promo_8: 17143.58
promo_1

Step 16
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.38228	valid_1's l2: 0.394232
[200]	training's l2: 0.36235	valid_1's l2: 0.37565
[300]	training's l2: 0.356636	valid_1's l2: 0.372139
[400]	training's l2: 0.35366	valid_1's l2: 0.371092
[500]	training's l2: 0.351632	valid_1's l2: 0.370594
mean_30_2017: 6121497.00
mean_14_2017: 2262742.05
mean_60_2017: 1867760.44
mean_7_2017: 834416.37
promo_15: 604583.12
mean_12_dow1_2017: 574421.55
mean_90_2017: 429160.17
mean_20_dow1_2017: 402780.46
mean_12_dow2_2017: 66331.27
day_1_2017: 59097.47
promo_14: 56902.00
item_class_nbr: 54213.69
mean_4_dow1_2017: 51201.49
promo_30_2017: 48569.58
mean_5_2017: 41283.94
mean_20_dow2_2017: 39939.95
item_family_nbr: 38748.52
mean_3_2017: 26881.67
promo_14_2017: 26303.54
mean_140_2017: 21356.83
promo_60_2017: 18601.37
promo_13: 18448.83
promo_140_2017: 17793.98
promo_7_2017: 17448.75
promo_10: 15584.93
promo_90_2017: 12964.14
perishable: 12462.80
promo_12: 11377.69
mean_

In [79]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb_v3.csv', float_format='%.4f', index=None)

Making submission...
